<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/AlgoTradingTest01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sourcing Alpaca Credentials

In [1]:
credentials = open("/content/drive/MyDrive/Colab Notebooks/alpaca.txt","r")
lines = credentials.readlines()
consumer_key = lines[0].rstrip().replace("alpaca_key=","")
consumer_secret = lines[1].rstrip().replace("alpaca_secret=","")
#print(consumer_secret)


In [2]:
!pip install alpaca-trade-api requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.9 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.5
    Uninstalling msgpack-1.0.5:
      Successfully uninstalled msgpack-1.0.5
  Attempting uninstall: PyYAML
   

In [95]:
import pandas as pd
from alpaca_trade_api import REST, Stream, TimeFrame,TimeFrameUnit
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta

API_KEY = consumer_key
API_SECRET = consumer_secret

def fetch_news_summary(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        summary = soup.find('p').text
        return summary.strip()
    except Exception as e:
        print(f"Error fetching news summary: {e}")
        return None

def get_news_data(ALPACA_API_KEY,ALPACA_SECRET_KEY,ticker):
    # Get News for last 24 hrs
    today = datetime.today()- timedelta(1)
    yesterday = today - timedelta(30)
    today = today.strftime('%Y-%m-%d')
    yesterday = yesterday.strftime('%Y-%m-%d')

    # Initialize Alpaca API
    # Get the news data
    rest_client = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    news_list = rest_client.get_news(ticker,yesterday, today)
    rest_client2 = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    bars = rest_client2.get_bars(ticker, TimeFrame(59, TimeFrameUnit.Minute), yesterday,today, adjustment='raw').df
    bars['time'] = bars.index.strftime('%Y-%m-%d %H:%M')
    bars['Ticker'] = ticker

    # Convert the news data into a pandas DataFrame
    data = {
        'Title': [],
        'time': [],
        'Source': [],
        'Url': [],
        'Summary': [],
        'Ticker' : []
    }

    for news in news_list:
        data['Title'].append(news.headline)
        data['time'].append(news.created_at.strftime('%Y-%m-%d %H:%M'))
        data['Source'].append(news.source)
        data['Url'].append(news.url)
        data['Summary'].append(fetch_news_summary(news.url))
        data['Ticker'].append(ticker)

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    bars["time"] = pd.to_datetime(bars["time"])

    return df,bars

if __name__ == "__main__":
    news_df,bars_data = get_news_data(API_KEY,API_SECRET,"TSLA")

In [96]:
merged_dataframe = pd.merge_asof(bars_data,news_df.sort_values('time'), on="time", by="Ticker",
                                 tolerance=pd.Timedelta("180m"))
news_df

,Title,time,Source,Url,Summary,Ticker
0,AI Frenzy Result – Perma Bulls Revered And Per...,2023-07-31 15:01:00,benzinga,https://www.benzinga.com/23/07/33472022/ai-fre...,"To gain an edge, this is what you need to know...",TSLA
1,"Exxon In Talks With Tesla, Ford, VW On Supplyi...",2023-07-31 17:47:00,benzinga,https://www.benzinga.com/news/23/07/33476063/e...,Never miss a trade again with the fastest news...,TSLA
2,10 Consumer Discretionary Stocks Whale Activit...,2023-07-31 17:35:00,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,TSLA
3,'Uber To Purchase 100 Tesla Model Ys For The J...,2023-07-31 17:18:00,benzinga,https://www.benzinga.com/news/23/07/33475622/u...,Never miss a trade again with the fastest news...,TSLA
4,Last Week's EV Updates Were All About Electric...,2023-07-31 15:24:00,benzinga,https://www.benzinga.com/23/07/33472770/last-w...,"Last week, Tesla Inc TSLA knocked on India’s d...",TSLA
5,Why Are Tesla Owners Selling Their Model 3s? H...,2023-07-29 17:04:00,benzinga,https://www.benzinga.com/news/23/07/33451883/w...,"Since launching in July 2017, the Model 3 from...",TSLA
6,Is Elon Musk's Success A Result Of Deception? ...,2023-07-29 04:37:00,benzinga,https://www.benzinga.com/news/23/07/33453340/i...,Facebook co-founder Dustin Moskovitz has cast ...,TSLA
7,Earnings And Non Farm Payroll Week Has Arrived...,2023-07-31 13:53:00,benzinga,https://www.benzinga.com/23/07/33470247/earnin...,It feels like every week has the ability to be...,TSLA
8,South Korea Emerges as Gateway for Chinese Fir...,2023-07-31 12:49:00,benzinga,https://www.benzinga.com/government/23/07/3346...,Chinese firms are investing in South Korea's b...,TSLA
9,Rivian Tweaks Warranty For Dual Motor System: ...,2023-07-31 09:59:00,benzinga,https://www.benzinga.com/news/23/07/33465397/r...,Rivian Automotive Inc RIVN has lowered its war...,TSLA
